# TP 3 - Probabilité d'exécution sur les marchés cryptos centralisés

Vous utiliserez une semaine de données L1 (prix/volume aux premières limites et transactions) sur les marchés centralisés (CEXs) Bitstamp, Kraken et Binance US et les paires BTC-USD et ETH-USD.

Installer `pyarrow` puis charger les données dans un `DataFrame` `pandas` en utilisant l'exemple suivant.

Conversion des timestamps des index au format `datetime` de `pandas`.

In [137]:
import numpy as np
from datetime import datetime
import pandas as pd

exchanges = ['binanceus', 'bitstamp', 'kraken']
coins = ['btcusd', 'ethusd']

dict_l1 = {}
dict_trades = {}

for exchange in exchanges:
    for coin in coins:

        df_l1 = pd.read_parquet(
            f"l1_{coin}_{exchange}.parquet",
            engine="pyarrow"
        )

        df_trades = pd.read_parquet(
            f"trades_{coin}_{exchange}.parquet",
            engine="pyarrow"
        )
        df_l1['imbalance'] = (df_l1['best_bid_qty'] - df_l1['best_ask_qty'])/(df_l1['best_bid_qty'] + df_l1['best_ask_qty'])
        df_l1['log_imbalance'] = np.log(df_l1['best_bid_qty']/df_l1['best_ask_qty'])
        df_l1['date'] = [datetime.fromtimestamp(t) for t in df_l1.index]
        df_trades['date'] = [datetime.fromtimestamp(t) for t in df_trades.index]
        dict_l1[f'{coin}_{exchange}'] = df_l1
        dict_trades[f'{coin}_{exchange}'] = df_trades


Dans les données L1, vous avez 4 colonnes avec les prix des bests bid et ask ainsi que les quantités disponibles.

In [138]:
dict_l1['btcusd_binanceus'].head()

,best_bid_price,best_ask_price,best_bid_qty,best_ask_qty,imbalance,log_imbalance,date
1.675210e+09,23127.29,23129.59,0.009625,0.059662,-0.72217,-1.824331,2023-02-01 01:00:00.014691
1.675210e+09,23127.29,23129.59,0.009625,0.059662,-0.72217,-1.824331,2023-02-01 01:00:00.014967
1.675210e+09,23127.29,23129.59,0.009625,0.059662,-0.72217,-1.824331,2023-02-01 01:00:00.014969
1.675210e+09,23127.29,23129.59,0.009625,0.059662,-0.72217,-1.824331,2023-02-01 01:00:00.014970
1.675210e+09,23127.29,23129.59,0.009625,0.059662,-0.72217,-1.824331,2023-02-01 01:00:00.014971


Dans les données des transactions, vous disposez des prix, des quantités et du côté (Bid/Ask) des transactions.

In [133]:
dict_trades['btcusd_binanceus'].head()

,price,size,maker_side
1.675210e+09,23127.29,0.009625,B
1.675210e+09,23127.11,0.043339,B
1.675210e+09,23127.08,0.011344,B
1.675210e+09,23127.07,0.011216,B
1.675210e+09,23127.05,0.011244,B


## 1. Microstructure du processus de rendements

##### Pour les horizons 1 seconde et 1 minute:

Calculer les séries de rendement du mid price pour les différentes paires (à horizon $h$ égal à 1 minute et 1 seconde) et faire un histogramme de la distribution pour chaque horizon $h$. Pour chaque $h$, calculer le kurtosis et le skewness, puis calculer la fonction de survie du rendement absolu $S$ définie par

$$S(r):=\mathbb{P}(|R|>r).$$

In [139]:
df_l1 = dict_l1['btcusd_binanceus']
df_trades = dict_trades['btcusd_binanceus']

df_l1['mid_price'] = (df_l1.best_bid_price + df_l1.best_bid_price) / 2
return_1s = (df_l1.mid_price.resample("1S").last().values - df_l1.mid_price.resample("1S").first().values) / df_l1.mid_price.resample("1S").first().values

TypeError: Only valid with DatetimeIndex, TimedeltaIndex or PeriodIndex, but got an instance of 'Float64Index'

Faire un plot de $S$ en échelle log-log pour chaque horizon $h$ et conclure. Les propriétés sont-elles similaires d'une bourse à une autre ?

In [ ]:
abs_returns_1s = np.abs(return_1s)

## 2. Calcul des probabilités d'exécution

### Préliminaire. Calcul de la taille du tick

Caluler la taille du tick de toutes les paires sur les trois marchés. On les stockera dans un dictionnaire pour pouvoir y accéder facilement.

In [140]:
dict_tick_size = {}

for exchange in exchanges:
    for coin in coins:

        diff = dict_l1[f'{coin}_{exchange}'].best_bid_price.diff().to_numpy()
        tick = np.around(np.min(diff[diff > 0]), 2)
        dict_tick_size[f'{coin}_{exchange}'] = tick

dict_tick_size

{'btcusd_binanceus': 0.01,
 'ethusd_binanceus': 0.01,
 'btcusd_bitstamp': 1.0,
 'ethusd_bitstamp': 0.1,
 'btcusd_kraken': 0.1,
 'ethusd_kraken': 0.01}

### a. Etude de la sensibilité selon la distance de placement

Considérons un agent qui place un ordre limite de taille infinitésimale à une distance $\delta\geq0$ ticks du BBO (Best Bid Offer). L'objectif de cette partie est d'étudier la probabilité $p(\delta)$ que l'ordre soit exécuté sous un horizon fixé $h$. On fixera $h$ égal à 10 secondes dans cette partie seulement.

Dans un premier temps, développer une méthode de calcul de cette probabilité pour une distance $\delta=0$ en réalisant un sampling des snapshots des carnets.

Par exemple: si on se place au bid, on peut échantillonner un carnet toutes les secondes puis placer un ordre "artificiel" à un prix $P_{bid}$ situé à une distance $\delta$ du best bid, puis considérer l'ordre comme exécuté à partir du moment où une transaction a eu lieu à un prix inférieur à $P_{bid}$. Enregistrer la valeur $X_i^\delta=1$ si l'ordre est exécuté, la valeur $X_i^\delta=0$ s'il ne l'est pas, puis calculer l'estimateur empirique sur l'ensemble des ordres:

$$\widehat{p(\delta)}=\frac{1}{N}\sum_{i=1}^NX_i^\delta.$$

In [146]:
def compute_proba_fill(delta: float=0, N: int=int(1e3), h: float=10, duration: int=10, venue: str='binanceus', coin: str='btcusd') -> float:
    somme = 0
    for i in range(N):
        df_l1 = dict_l1[coin + '_' + venue]
        df_trades = dict_trades[coin + '_' + venue]
        price = df_l1.iloc[duration*i].best_bid_price - delta
        ts = df_l1.iloc[duration*i].name

        if (df_trades[(df_trades.index > ts) & (df_trades.index <= ts + h)]['price'] < price).any():

            somme += 1

    return somme / N

Dans un second temps, faire varier la distance $\delta$ de placement pour obtenir une courbe de probabilité d'exécution. Avec quelle classe de fonction paramétrique modéliseriez-vous la fonction $p(\delta)$ ?

In [147]:
compute_proba_fill()

0.614

In [148]:
deltas = np.linspace(0, 10*dict_tick_size['btcusd_binanceus'], 10)

for delta in deltas:

    print(compute_proba_fill(delta=delta))

0.614
0.606
0.603
0.599
0.597
0.595
0.594
0.594
0.592
0.591


error_bar

### b. Etude de la sensibilité selon les états du marché

Construire des variables qui caractérisent l'état du marché au moment du placement (par exemple, l'imbalance au BBO ou le spread bid-ask). Conditionner la probabilité par rapport à chaque état pour faire apparaître des relations. On se restreindra à une 1 dimension: créer des "buckets", par exemple \[-1, -0.5, 0, 0.5, 1\] pour l'imbalance. Commenter les résultats.

## 3. Développement d'une stratégie de trading

##### Cette partie est optionnelle et à réaliser à la maison. Elle sera notée sur 5 et apportera UNIQUEMENT des points bonus pour la note finale.

Développer une stratégie utilisant les deux exercices précédents sur l’un des deux sujets suivants et la tester :

- _Exécution optimale_ : exécution stratégique via Almgren-Chriss puis tactique via les probabilités de premier temps de passage (découper la courbe d’exécution en sous-périodes de 5 secondes par exemple et utiliser un algorithme d’implementation shortfall : choisir entre poster un ordre limite et un ordre marché basé sur la probabilité d’exécution).


- _Market making_ : utiliser une stratégie simple de type Avellaneda-Stoïkov mais y inclure un modèle de probabilité d'exécution et un signal de changement de régime de marché à définir (un jeu de paramètres par régime est souhaitable) ; focus sur la gestion du risque d’inventaire via une estimation précise des probabilités d’exécution.

#### Vous pourrez utiliser la littérature de votre choix et même en extraire des idées. Les citations bibliographiques sont bienvenues pour appuyer votre raisonnement. La performance/robustesse de votre stratégie devra être évaluée en utilisant des données Out-of-Sample! (Estimer les paramètres de votre modèle sur une période et tester la stratégie sur une autre période est obligatoire...)

**Cette dernière partie est à rendre au plus tard le 12 mars à 20h.**

t.fabre@sunzulab.com